<a href="https://colab.research.google.com/github/edufsi/Projeto-Desafio/blob/Eduardo/Limpeza_Dados_Exportacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import pandas as pd
import numpy as np

In [49]:
#Importar dados:
url_commodities = "https://raw.githubusercontent.com/edufsi/Projeto-Desafio/main/Dados/UNdata_Export_20240412_201846138.csv"
url_parceiros = "https://raw.githubusercontent.com/edufsi/Projeto-Desafio/main/Dados/SYB66_330_202310_Major%20Trading%20Partners.csv"
url_exportacoes = "https://raw.githubusercontent.com/edufsi/Projeto-Desafio/main/Dados/SYB66_123_202310_Total%20Imports%20Exports%20and%20Balance%20of%20Trade.csv"
url_cambio = "https://raw.githubusercontent.com/edufsi/Projeto-Desafio/main/Dados/USD_BRL_Historical_Data.csv"


dados_commodities = pd.read_csv(url_commodities)
dados_parceiros = pd.read_csv(url_parceiros, encoding = "ISO-8859-1")
dados_exportacoes = pd.read_csv(url_exportacoes, encoding = "ISO-8859-1")
dados_cambio = pd.read_csv(url_cambio)

# LIMPEZA DOS DADOS DE EXPORTAÇÃO

In [ ]:
# VISÃO GERAL
print(dados_exportacoes.head())

In [ ]:
print(dados_exportacoes.info())

In [52]:
# Os titulos das colunas estao errados. Eles estao na segunda coluna. Na primeira temos um titulo
# Podemos usar iloc para selecionar linhas. Passando um intervalo, uma lista ou um numero retornamos as linhas correspondentes


# Vamos tomar a primeira linha e defini-la como nossas colunas
dados_exportacoes.columns = dados_exportacoes.iloc[0]


# Agora que os titulos ja estao corretos, podemos excluir essa primeira linha que continha os títulos
dados_exportacoes.drop(0, inplace=True)


In [ ]:
# Ainda há uma coluna "errada" no entanto. No arquivo original, foram associados valores para representar as regioes/paises/areas e há uma coluna sem titulo para a descricao
# Seria melhor aqui ter apenas uma coluna com os nomes das regiões, áreas e dos países. Temos que alter entao o nome da segunda coluna, essa sem titulo


lista = [x for x in dados_exportacoes.columns]
lista[1] = "Region/Country" # Substituindo o nome da coluna com nan por "Region/Country" (segunda coluna)

dados_exportacoes.columns = lista

print(dados_exportacoes.info())

In [54]:
# Agora podemos retirar as colunas inúteis
# Podemos retirar a coluna "System of trade footnote", pois ela nao tem nenhum valor nao nulo
# Tambem podemos tirar a coluna de codificacao das regioes, ja que iremos nos referir a elas pelo nome
# A coluna Source também pode ser retirada pois apresenta apenas um valor
# A coluna Footnotes também pode ser retirada, assim como a coluna de system os trade
dados_exportacoes.drop(["System of trade footnote", "Region/Country/Area", "Source", "Footnotes", "System of trade"], axis=1, inplace=True)



In [55]:
# A coluna de valores esta como tipo objeto. Precisamos tranformá-la em numero. Nao temos numeros quebrados, podemos transformá-la em numeros inteiros
# Precisamos retirar as virgulas dos numeros, para entao transforma-los em inteiros. Vamos transformar em inteiros de 8 bytes para ter certeza de não ter problemas de overflow

dados_exportacoes['Value'] = dados_exportacoes['Value'].apply(lambda x: x.replace(',', '')).astype(np.int64)

In [56]:
# Finalmente temos um dataframe em que todas as colunas dizem algo util


# Vamos agora tirar as regiões e uniões para o lado, deixando só os países

regioes = ['Total, all countries or areas', 'Africa', 'Northern Africa',
 'Sub-Saharan Africa', 'Eastern Africa', 'Middle Africa', 'Southern Africa',
 'Western Africa', 'Americas', 'Northern America',
 'Latin America & the Caribbean', 'Caribbean', 'Central America',
 'South America', 'Asia', 'Central Asia', 'Eastern Asia', 'South-eastern Asia',
 'Southern Asia', 'Western Asia', 'Europe', 'Eastern Europe',
 'Northern Europe', 'Southern Europe', 'Western Europe', 'Oceania',
 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia']



exportacoes_regioes = dados_exportacoes.query('`Region/Country` in @regioes')
exportacoes_paises = dados_exportacoes.query('`Region/Country` not in @regioes')

In [57]:
# Terminamos com 2 dataframe: um para as regiões e agrupamentos e outro com cada país individual

In [58]:
exportacoes_regioes

,Region/Country,Year,Series,Value
1,"Total, all countries or areas",1995,Imports CIF (millions of US dollars),5099057
2,"Total, all countries or areas",2005,Imports CIF (millions of US dollars),10577013
3,"Total, all countries or areas",2010,Imports CIF (millions of US dollars),15275115
4,"Total, all countries or areas",2015,Imports CIF (millions of US dollars),16482346
5,"Total, all countries or areas",2020,Imports CIF (millions of US dollars),17598001
...,...,...,...,...
626,Polynesia,2010,Balance imports/exports (millions of US dollars),-2095
627,Polynesia,2015,Balance imports/exports (millions of US dollars),-2058
628,Polynesia,2020,Balance imports/exports (millions of US dollars),-2281
629,Polynesia,2021,Balance imports/exports (millions of US dollars),-2603


In [59]:
exportacoes_paises

,Region/Country,Year,Series,Value
631,Afghanistan,2010,Imports CIF (millions of US dollars),5154
632,Afghanistan,2015,Imports CIF (millions of US dollars),7723
633,Afghanistan,2020,Imports CIF (millions of US dollars),8266
634,Afghanistan,2021,Imports CIF (millions of US dollars),5505
635,Afghanistan,2022,Imports CIF (millions of US dollars),4957
...,...,...,...,...
5049,Zimbabwe,2010,Balance imports/exports (millions of US dollars),-2653
5050,Zimbabwe,2015,Balance imports/exports (millions of US dollars),-2642
5051,Zimbabwe,2020,Balance imports/exports (millions of US dollars),-653
5052,Zimbabwe,2021,Balance imports/exports (millions of US dollars),-1541
